<a href="https://colab.research.google.com/github/jdavidram/gestoresRCD/blob/appDash/ModeloPredictivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# librerías necesarias
import pandas as pd

In [2]:
# cargar el conjunto de datos
data = pd.read_csv('itemsEjec.csv')

In [3]:
#reemplazar la columna municipios por valores numéricos
municipios_unicos = data["Municipio"].unique()
mapeo_municipios = {mun: i+1 for i, mun in enumerate(municipios_unicos)}
data["Municipio"] = data["Municipio"].map(mapeo_municipios)
print("\nMapa usado:", municipios_unicos)

#reemplazar la columna subregión por valores numéricos
Subregiones_unicas = data["Subregion"].unique()
mapeo_subregiones = {subr: i+1 for i, subr in enumerate(Subregiones_unicas)}
data["Subregion"] = data["Subregion"].map(mapeo_subregiones)
print("\nMapa usado:", Subregiones_unicas)

# Reemplazar True/False por 1/0 en todas las columnas booleanas
bool_cols = data.select_dtypes(include=["bool"]).columns
data[bool_cols] = data[bool_cols].astype(int)

# seleccionamos las columnas para el modelo
data = data[["Municipio", "Latitud_Y", "Longitud_X", "areaPavimento", "areaConcreto", "areaRoca", "areaTierras", "Subregion"]]

# dataframe listo para el modelo de bosques aleatorios
print(data)


Mapa usado: ['BELLO' 'ITUANGO' 'CAREPA' 'TURBO' 'CHIGORODO' 'ARBOLETES' 'NECOCLI'
 'CAROLINA DEL PRINCIPE' 'MEDELLIN' 'ENVIGADO' 'GIRARDOTA' 'SABANETA'
 'GOMEZ PLATA' 'REMEDIOS' 'SAN JERONIMO' 'EL RETIRO'
 'SANTAFE DE ANTIOQUIA' 'BELMIRA' 'SOPETRAN' 'COPACABANA' 'CALDAS'
 'ITAGUI' 'LA ESTRELLA' 'GUARNE' 'BETULIA' 'VENECIA' 'AMAGA' 'CONCORDIA'
 'TARSO' 'TAMESIS' 'PUEBLORRICO' 'SANTA BARBARA'
 'SAN PEDRO DE LOS MILAGROS' 'SAN ANDRES DE CUERQUIA' 'AMALFI' 'LIBORINA'
 'SAN ROQUE' 'DONMATIAS' 'YARUMAL' 'SAN CARLOS' 'SANTA ROSA DE OSOS'
 'BARBOSA' 'SAN RAFAEL' 'SONSON' 'URRAO' 'BETANIA' 'CIUDAD BOLIVAR'
 'LA CEJA' 'HISPANIA' 'CARAMANTA' 'JARDIN' 'GUATAPE' 'LA UNION'
 'ANGELOPOLIS' 'TITIRIBI' 'SAN LUIS' 'SALGAR' 'JERICO' 'ABEJORRAL'
 'CONCEPCION' 'MONTEBELLO' 'VALPARAISO' 'FREDONIA' 'EL PEÑOL'
 'SAN VICENTE' 'YALI' 'PUERTO BERRIO' 'TOLEDO' 'YOLOMBO' 'MACEO'
 'CAÑASGORDAS' 'ENTRERRIOS' 'SAN JOSE DE LA MONTAÑA' 'CARACOLI' 'DABEIBA'
 'OLAYA' 'PUERTO NARE' 'COCORNA' 'VEGACHI' 'FRONTINO' 'SANTO D

In [5]:
# importar librerías necesarias
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix,
                             ConfusionMatrixDisplay, f1_score, roc_auc_score)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Encontrar mejor modelo para "areaPavimento"
# Datos
X = data[["Municipio", "Latitud_Y", "Longitud_X", "Subregion"]]
y = data["areaPavimento"]

# Dividir datos manteniendo proporciones
Xtr, Xte, ytr, yte = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

# Definimos búsqueda de hiperparámetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'criterion': ['gini', 'entropy']
}

# Modelo base
rf = RandomForestClassifier(random_state=42)

# GridSearchCV para optimizar
grid_search = GridSearchCV(
    rf,
    param_grid,
    scoring='f1',  # optimizamos F1 para balancear precisión y recall
    cv=5,
    n_jobs=-1,
    verbose=1
)

# Entrenar
grid_search.fit(Xtr, ytr)

# Mejor combinación encontrada
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor F1 en CV:", grid_search.best_score_)

# Entrenar modelo final con los mejores parámetros
mejor_modelo = grid_search.best_estimator_
y_pred = mejor_modelo.predict(Xte)

# Evaluación
print("\nMatriz de confusión:")
print(confusion_matrix(yte, y_pred))
print("\nReporte de clasificación:")
print(classification_report(yte, y_pred))


In [9]:
# modelo definitivo para "areaPavimento"

# Datos
X = data[["Municipio", "Latitud_Y", "Longitud_X", "Subregion"]]
y = data["areaPavimento"]

# Dividir datos manteniendo proporciones
Xtr, Xte, ytr, yte = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

# Entrenar modelo final con los mejores parámetros
mejor_modelo_pavimento = RandomForestClassifier(
    criterion='gini',
    max_depth=None,
    max_features='sqrt',
    min_samples_leaf=1,
    min_samples_split=10,
    n_estimators=100,
    random_state=42
)

mejor_modelo_pavimento.fit(Xtr, ytr)

# Predicciones
y_pred = mejor_modelo_pavimento.predict(Xte)

# Evaluación
print("\nMatriz de confusión:")
print(confusion_matrix(yte, y_pred))

print("\nReporte de clasificación:")
print(classification_report(yte, y_pred))



Matriz de confusión:
[[1897    2]
 [   1 1569]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1899
           1       1.00      1.00      1.00      1570

    accuracy                           1.00      3469
   macro avg       1.00      1.00      1.00      3469
weighted avg       1.00      1.00      1.00      3469



In [ ]:
# Encontrar mejor modelo para "areaConcreto"
# Datos
X = data[["Municipio", "Latitud_Y", "Longitud_X", "Subregion"]]
y = data["areaConcreto"]

# Dividir datos manteniendo proporciones
Xtr, Xte, ytr, yte = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

# Definimos búsqueda de hiperparámetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'criterion': ['gini', 'entropy']
}

# Modelo base
rf = RandomForestClassifier(random_state=42)

# GridSearchCV para optimizar
grid_search = GridSearchCV(
    rf,
    param_grid,
    scoring='f1',  # optimizamos F1 para balancear precisión y recall
    cv=5,
    n_jobs=-1,
    verbose=1
)

# Entrenar
grid_search.fit(Xtr, ytr)

# Mejor combinación encontrada
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor F1 en CV:", grid_search.best_score_)

# Entrenar modelo final con los mejores parámetros
mejor_modelo = grid_search.best_estimator_
y_pred = mejor_modelo.predict(Xte)

# Evaluación
print("\nMatriz de confusión:")
print(confusion_matrix(yte, y_pred))
print("\nReporte de clasificación:")
print(classification_report(yte, y_pred))

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Mejores parámetros: {'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Mejor F1 en CV: 0.9993303569343276

Matriz de confusión:
[[1548    0]
 [   0 1921]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1548
           1       1.00      1.00      1.00      1921

    accuracy                           1.00      3469
   macro avg       1.00      1.00      1.00      3469
weighted avg       1.00      1.00      1.00      3469



In [10]:
# modelo definitivo para "areaConcreto"

# Datos
X = data[["Municipio", "Latitud_Y", "Longitud_X", "Subregion"]]
y = data["areaConcreto"]

# Dividir datos manteniendo proporciones
Xtr, Xte, ytr, yte = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

# Entrenar modelo final con los mejores parámetros
mejor_modelo_concreto = RandomForestClassifier(
    criterion='entropy',
    max_depth=None,
    max_features='sqrt',
    min_samples_leaf=1,
    min_samples_split=2,
    n_estimators=100,
    random_state=42
)

mejor_modelo_concreto.fit(Xtr, ytr)

# Predicciones
y_pred = mejor_modelo_concreto.predict(Xte)

# Evaluación
print("\nMatriz de confusión:")
print(confusion_matrix(yte, y_pred))

print("\nReporte de clasificación:")
print(classification_report(yte, y_pred))



Matriz de confusión:
[[1548    0]
 [   0 1921]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1548
           1       1.00      1.00      1.00      1921

    accuracy                           1.00      3469
   macro avg       1.00      1.00      1.00      3469
weighted avg       1.00      1.00      1.00      3469



In [ ]:
# Encontrar mejor modelo para "areaRoca"
# Datos
X = data[["Municipio", "Latitud_Y", "Longitud_X", "Subregion"]]
y = data["areaRoca"]

# Dividir datos manteniendo proporciones
Xtr, Xte, ytr, yte = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

# Definimos búsqueda de hiperparámetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'criterion': ['gini', 'entropy']
}

# Modelo base
rf = RandomForestClassifier(random_state=42)

# GridSearchCV para optimizar
grid_search = GridSearchCV(
    rf,
    param_grid,
    scoring='f1',  # optimizamos F1 para balancear precisión y recall
    cv=5,
    n_jobs=-1,
    verbose=1
)

# Entrenar
grid_search.fit(Xtr, ytr)

# Mejor combinación encontrada
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor F1 en CV:", grid_search.best_score_)

# Entrenar modelo final con los mejores parámetros
mejor_modelo = grid_search.best_estimator_
y_pred = mejor_modelo.predict(Xte)

# Evaluación
print("\nMatriz de confusión:")
print(confusion_matrix(yte, y_pred))
print("\nReporte de clasificación:")
print(classification_report(yte, y_pred))

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Mejores parámetros: {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 300}
Mejor F1 en CV: 0.9992186874759245

Matriz de confusión:
[[1549    0]
 [   0 1920]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1549
           1       1.00      1.00      1.00      1920

    accuracy                           1.00      3469
   macro avg       1.00      1.00      1.00      3469
weighted avg       1.00      1.00      1.00      3469



In [11]:
# modelo definitivo para "areaRoca"

# Datos
X = data[["Municipio", "Latitud_Y", "Longitud_X", "Subregion"]]
y = data["areaRoca"]

# Dividir datos manteniendo proporciones
Xtr, Xte, ytr, yte = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

# Entrenar modelo final con los mejores parámetros
mejor_modelo_roca = RandomForestClassifier(
    criterion='gini',
    max_depth=None,
    max_features='sqrt',
    min_samples_leaf=2,
    min_samples_split=2,
    n_estimators=300,
    random_state=42
)

mejor_modelo_roca.fit(Xtr, ytr)

# Predicciones
y_pred = mejor_modelo_roca.predict(Xte)

# Evaluación
print("\nMatriz de confusión:")
print(confusion_matrix(yte, y_pred))

print("\nReporte de clasificación:")
print(classification_report(yte, y_pred))



Matriz de confusión:
[[1549    0]
 [   0 1920]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1549
           1       1.00      1.00      1.00      1920

    accuracy                           1.00      3469
   macro avg       1.00      1.00      1.00      3469
weighted avg       1.00      1.00      1.00      3469



In [ ]:
# Encontrar mejor modelo para "areaTierras"
# Datos
X = data[["Municipio", "Latitud_Y", "Longitud_X", "Subregion"]]
y = data["areaTierras"]

# Dividir datos manteniendo proporciones
Xtr, Xte, ytr, yte = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

# Definimos búsqueda de hiperparámetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    'criterion': ['gini', 'entropy']
}

# Modelo base
rf = RandomForestClassifier(random_state=42)

# GridSearchCV para optimizar
grid_search = GridSearchCV(
    rf,
    param_grid,
    scoring='f1',  # optimizamos F1 para balancear precisión y recall
    cv=5,
    n_jobs=-1,
    verbose=1
)

# Entrenar
grid_search.fit(Xtr, ytr)

# Mejor combinación encontrada
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor F1 en CV:", grid_search.best_score_)

# Entrenar modelo final con los mejores parámetros
mejor_modelo = grid_search.best_estimator_
y_pred = mejor_modelo.predict(Xte)

# Evaluación
print("\nMatriz de confusión:")
print(confusion_matrix(yte, y_pred))
print("\nReporte de clasificación:")
print(classification_report(yte, y_pred))

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Mejores parámetros: {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Mejor F1 en CV: 0.9995374375204079

Matriz de confusión:
[[1617    1]
 [   5 1846]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1618
           1       1.00      1.00      1.00      1851

    accuracy                           1.00      3469
   macro avg       1.00      1.00      1.00      3469
weighted avg       1.00      1.00      1.00      3469



In [12]:
# modelo definitivo para "areaTierras"

# Datos
X = data[["Municipio", "Latitud_Y", "Longitud_X", "Subregion"]]
y = data["areaTierras"]

# Dividir datos manteniendo proporciones
Xtr, Xte, ytr, yte = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

# Entrenar modelo final con los mejores parámetros
mejor_modelo_tierra = RandomForestClassifier(
    criterion='gini',
    max_depth=None,
    max_features='sqrt',
    min_samples_leaf=1,
    min_samples_split=5,
    n_estimators=100,
    random_state=42
)

mejor_modelo_tierra.fit(Xtr, ytr)

# Predicciones
y_pred = mejor_modelo_tierra.predict(Xte)

# Evaluación
print("\nMatriz de confusión:")
print(confusion_matrix(yte, y_pred))

print("\nReporte de clasificación:")
print(classification_report(yte, y_pred))



Matriz de confusión:
[[1617    1]
 [   5 1846]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1618
           1       1.00      1.00      1.00      1851

    accuracy                           1.00      3469
   macro avg       1.00      1.00      1.00      3469
weighted avg       1.00      1.00      1.00      3469



In [ ]:

municipios_unicos = municipios_unicos.tolist()
subregiones_unicas = Subregiones_unicas.tolist()


In [20]:
# Instalar dependencias si aún no están
!pip install ipyleaflet ipywidgets --quiet

# Importar módulos
from ipyleaflet import Map, Marker
from ipywidgets import Output, VBox, Button

# Centro de Antioquia
center_lat, center_lon = 6.5569, -75.8281

# Crear mapa
m = Map(center=(center_lat, center_lon), zoom=8)

# Variable para guardar coordenadas
selected_coords = {}

# Salida para mostrar mensajes
out = Output()

# Función para manejar clic en el mapa
def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        lat, lon = kwargs.get('coordinates')
        selected_coords["lat"] = lat
        selected_coords["lon"] = lon
        with out:
            out.clear_output()
            print(f"📍 Coordenadas guardadas: {lat:.5f}, {lon:.5f}")
        # Limpiar marcadores anteriores y agregar uno nuevo
        m.layers = m.layers[:1]
        m.add_layer(Marker(location=(lat, lon)))

# Botón para mostrar coordenadas guardadas
btn_show = Button(description="Mostrar coordenadas guardadas")
def show_coords(btn):
    with out:
        out.clear_output()
        if selected_coords:
            print(f"✅ Coordenadas actuales: {selected_coords}")
        else:
            print("❗ Aún no has seleccionado ninguna ubicación.")
btn_show.on_click(show_coords)

# Vincular el evento de clic
m.on_interaction(handle_click)

# Mostrar todo junto
VBox([m, btn_show, out])



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.4 MB/s eta 0:00:00


In [21]:
# Predecir nuevos datos

nuevos_datos = pd.DataFrame([{
    "Municipio": "MEDELLIN",
    "Latitud_Y": 6.2476,
    "Longitud_X": -75.5658,
    "Subregion": "VALLE DE ABURRA"
}])

nuevos_datos["Municipio"] = nuevos_datos["Municipio"].apply(
    lambda x: municipios_unicos.index(x) + 1
)

nuevos_datos["Subregion"] = nuevos_datos["Subregion"].apply(
    lambda x: subregiones_unicas.index(x) + 1
)

print(nuevos_datos)

# Evaluar en el modelo

prediccion = mejor_modelo_pavimento.predict(nuevos_datos)
print("Predicción:", prediccion[0])

prediccion = mejor_modelo_concreto.predict(nuevos_datos)
print("Predicción:", prediccion[0])

prediccion = mejor_modelo_roca.predict(nuevos_datos)
print("Predicción:", prediccion[0])

prediccion = mejor_modelo_tierra.predict(nuevos_datos)
print("Predicción:", prediccion[0])

   Municipio  Latitud_Y  Longitud_X  Subregion
0          9     6.2476    -75.5658          1
Predicción: 1
Predicción: 1
Predicción: 1
Predicción: 1
